In [1]:
# Requirement: 'https://www.kaggle.com/c/fake-news/'

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import accuracy_score


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot


import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [5]:
df = pd.read_csv('dataset/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.shape

(20800, 5)

In [6]:
# Check NaN values
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
# drop the records with NaN values
df = df.dropna()

In [12]:
# Prepare the X nd Y set
X = df.iloc[:,:-1]
Y = df.label

### Preprocess the input

In [16]:
# Vocab size
vocab_size = 5000

In [21]:
messages = X.copy()
messages.reset_index(inplace=True)

#### 1. Stop words removal & stemming

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/himanshubhusan.rath/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
# Function to prepare a corpus for the given feature
# Preprocessing like Stop word removal, non-characters removal, making to lower case etc are performed
def preprocess_text_input(data):
    
    ps = PorterStemmer()
    corpus = []
    
    for i in range(0, len(data)):
        # Remove all characters apart from a-z and A-Z
        text = re.sub('[^a-zA-Z]',' ',data[i])
        # Convert to lower case
        text = text.lower()
        # Get words from the sentences by splitting
        words = text.split()
        # filter stop words and perform stemming
        words = [ps.stem(word) for word in words if not word in stopwords.words('english')]
        # Join the words to form a sentence again
        text = ' '.join(words)

        corpus.append(text)
        
    return corpus


In [34]:
# preprocess the 'title' feature
corp = preprocess_text_input(messages['title'])
corp

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

#### 2. One-hot representation

In [36]:
onehot_repr = [one_hot(words, vocab_size) for words in corp]
onehot_repr

[[2167, 3071, 2639, 4352, 3278, 1627, 2198, 256, 4850, 2140],
 [573, 4437, 4405, 1668, 4645, 1368, 3298],
 [116, 3967, 4741, 112],
 [1274, 1501, 4730, 4791, 4362, 4507],
 [4874, 4645, 236, 4471, 723, 631, 4645, 2240, 4926, 4272],
 [231,
  3046,
  4321,
  4800,
  103,
  4101,
  2524,
  4427,
  205,
  2710,
  1900,
  2921,
  4409,
  958,
  3298],
 [857, 669, 443, 2100, 1010, 3996, 3726, 2932, 4072, 1458, 1696],
 [4056, 4389, 2877, 4327, 4679, 3538, 4101, 4139, 4072, 1458, 1696],
 [141, 3354, 539, 2514, 3312, 4364, 3313, 2585, 4101, 4680],
 [2139, 4268, 2679, 2677, 4299, 2821, 2248, 3559],
 [3669, 265, 2088, 797, 1912, 1156, 2940, 1858, 4244, 3264, 1328],
 [4791, 878, 3278, 4364, 4101, 4679],
 [4677, 2608, 4339, 4756, 2972, 494, 4245, 2364, 3180],
 [835, 4109, 170, 954, 2100, 2317, 678, 4072, 1458, 1696],
 [3701, 349, 242, 456, 253, 4072, 1458, 1696],
 [362, 4502, 1105, 1710, 577, 647, 1726, 3855, 331, 3389],
 [3436, 2203, 4437],
 [2979, 4689, 1316, 3830, 4101, 2837, 9, 3298],
 [2603, 812

#### 3. Sequence padding

In [37]:
# Step-1 : Padding sequences to make all sentences equal in number of words
max_sentence_len = 20
embeded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=max_sentence_len)
embeded_docs

array([[   0,    0,    0, ...,  256, 4850, 2140],
       [   0,    0,    0, ..., 4645, 1368, 3298],
       [   0,    0,    0, ..., 3967, 4741,  112],
       ...,
       [   0,    0,    0, ..., 4072, 1458, 1696],
       [   0,    0,    0, ..., 1628, 3974, 3857],
       [   0,    0,    0, ..., 3822, 4553, 2415]], dtype=int32)

## Define the model

In [38]:
embedding_feature_count = 40

model = Sequential()
# Add the embedding layer
model.add(Embedding(vocab_size, embedding_feature_count, input_length=max_sentence_len))
# Add LSTM layer (one layer with 100 LSTM cells) -- CHECK THIS
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


## Model training

In [39]:
# Prepare the training data
X_final = np.array(embeded_docs)
Y_final = np.array(Y)

In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.33, random_state=42)

In [42]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 24s 2ms/sample - loss: 0.3301 - accuracy: 0.8501 - val_loss: 0.1979 - val_accuracy: 0.9163
Epoch 2/10
12250/12250 [==============================] - 20s 2ms/sample - loss: 0.1397 - accuracy: 0.9442 - val_loss: 0.2258 - val_accuracy: 0.9152
Epoch 3/10
12250/12250 [==============================] - 21s 2ms/sample - loss: 0.0978 - accuracy: 0.9647 - val_loss: 0.2413 - val_accuracy: 0.9097
Epoch 4/10
12250/12250 [==============================] - 21s 2ms/sample - loss: 0.0720 - accuracy: 0.9744 - val_loss: 0.2911 - val_accuracy: 0.9125
Epoch 5/10
12250/12250 [==============================] - 20s 2ms/sample - loss: 0.0602 - accuracy: 0.9798 - val_loss: 0.3392 - val_accuracy: 0.9092
Epoch 6/10
12250/12250 [==============================] - 20s 2ms/sample - loss: 0.0420 - accuracy: 0.9860 - val_loss: 0.3336 - val_accuracy: 0.9041
Epoch 7/10
12250/12250 [==============================] -

## Performance Metrics

In [47]:
pred = model.predict_classes(X_test)

In [56]:
confusion_matrix(pred,Y_test)

array([[3110,  285],
       [ 309, 2331]])

In [58]:
accuracy_score(pred, Y_test)

0.9015741507870754